In [1]:
from pathlib import Path
from typing import Tuple, Union, List, Dict, Iterable, Optional

import torch
from tqdm.notebook import tqdm

from decomposer import Decomposer, DecomposerConfig
from recomposer import Recomposer, RecomposerConfig
# from evaluations.helpers import GroundedWord, load_recomposers_en_masse
# from evaluations.clustering import graph_en_masse
# from evaluations.euphemism import cherry_words

In [12]:
from dataclasses import dataclass

BASE_DIR = Path.home() / 'Research/congressional_adversary/results'
sup_PE = torch.load(BASE_DIR / 'SGNS deno/pretrained super large/init.pt')['model']
WID = sup_PE.word_to_id
print(f'Vocab size = {len(WTI):,}')


sub_PE = torch.load(BASE_DIR / 'bill topic/pretrained subset/init.pt')['model']
sub_PE_WID = sub_PE.word_to_id
sub_PE_GD = sub_PE.grounding
del sub_PE


def nearest_neighbors(
        query: Union[str, np.ndarray],
        embed: np.ndarray,
        top_k: int = 10
        ) -> None:

    query_vec = vec(query, embed)
    distances = [
        distance.cosine(query_vec, neighbor_vec)
        for neighbor_vec in embed]
    neighbors = np.argsort(distances)

    
@dataclass
class GroundedWord():
    word: str

    def __post_init__(self) -> None:
        self.word_id: int = WTI[self.word]
        metadata = sub_PE_GD[self.word]
        self.freq: int = metadata['freq']
        self.R_ratio: float = metadata['R_ratio']
        self.majority_deno: int = metadata['majority_deno']
            
#         self.sub_PE_neighbors: List[str] = nearest(, sub_PE)

    def __str__(self) -> str:
        return str(vars(self))
    
    
capitalism: List[GroundedWord] = []
socialism: List[GroundedWord] = []
for word in sub_PE_WID.keys():
    ratio = sub_PE_GD[word]['R_ratio']
    freq = sub_PE_GD[word]['freq']
    word = GroundedWord(word)
    if ratio < 0.2 and freq > 100:  # 0.2:
        socialism.append(word)
    elif ratio > 0.8 and freq > 100:  # 0.8:
        capitalism.append(word)

print(
    f'{len(capitalism)} capitalists\n'
    f'{len(socialism)} socialists')
polarization = capitalism + socialism

Vocab size = 111,386
29 capitalists
64 socialists


In [6]:
import numpy as np

def get_embed(model: Decomposer) -> np.ndarray:
    return model.embedding.weight.detach().cpu().numpy()


def load(
        path: Path,
        match_vocab: bool = False,
        device: str = 'cpu'
        ) -> np.ndarray:
    model = torch.load(path, map_location=device)['model']
    try:
        assert model.word_to_id == WTI
    except AssertionError:
        print(f'Vocabulary mismatch: {path}')
        print(f'Vocab size = {len(model.word_to_id)}')
        if match_vocab:
            raise RuntimeError
        else:
            return None
    return get_embed(model)


def load_decomposers_en_masse(
        in_dirs: Union[Path, List[Path]],
        patterns: Union[str, List[str]]
        ) -> Tuple[Dict[str, np.ndarray], ...]:
    if not isinstance(in_dirs, List):
        in_dirs = [in_dirs, ]
    if not isinstance(patterns, List):
        patterns = [patterns, ]
    checkpoints: List[Path] = []
    for in_dir in in_dirs:
        for pattern in patterns:
            checkpoints += list(in_dir.glob(pattern))
    if len(checkpoints) == 0:
        raise FileNotFoundError('No model with path pattern found at in_dir?')

    models = {
#         'pretrained superset': load(BASE_DIR / 'bill topic/pretrained superset/init.pt'),
#         'pretrained subset': load(BASE_DIR / 'bill topic/pretrained subset/init.pt')
    }
    for path in tqdm(checkpoints):
        tqdm.write(f'Loading {path}')
        embed = load(path) 
        if embed is None:
            continue
        name = path.parent.name
        models[name] = embed
    return models

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

sns.set()

def plot(
        coordinates: np.ndarray,
        words: List[GroundedWord],
        path: Path
        ) -> None:
    fig, ax = plt.subplots(figsize=(15, 10))
#     skew = [w.R_ratio for w in words]
#     freq = [w.freq for w in words]
    sns.scatterplot(
        coordinates[:, 0], coordinates[:, 1],
#         hue=skew, palette='coolwarm',  # hue_norm=(0, 1),
#         size=freq, sizes=(200, 1000),
        legend=None, ax=ax)
    for coord, w in zip(coordinates, words):
        ax.annotate(w.word, coord, fontsize=20)
    with open(path, 'wb') as file:
        fig.savefig(file, dpi=300)
    plt.close(fig)


def plot_categorical(
        coordinates: np.ndarray,
        words: List[GroundedWord],
        path: Path,
        fancy: bool = True
        ) -> None:
    if fancy:
        fig, ax = plt.subplots(figsize=(20, 10))
        categories = [w.majority_deno for w in words]
        freq = [w.freq for w in words]
        sns.scatterplot(
            coordinates[:, 0], coordinates[:, 1],
            hue=categories, palette='muted', hue_norm=(0, 1),
            size=freq, sizes=(200, 1000),
            legend='brief', 
            ax=ax)
        chartBox = ax.get_position()
        ax.set_position(  # adjust legend
            [chartBox.x0, chartBox.y0, chartBox.width * 0.6, chartBox.height])
        ax.legend(loc='upper center', bbox_to_anchor=(1.45, 0.8), ncol=1)
    else:
        fig, ax = plt.subplots(figsize=(20, 10))
        freq = [w.freq for w in words]
        sns.scatterplot(
            coordinates[:, 0], coordinates[:, 1], ax=ax)

    for coord, w in zip(coordinates, words):
        ax.annotate(w.word, coord, fontsize=12)
    with open(path, 'wb') as file:
        fig.savefig(file, dpi=300)
    plt.close(fig)


def graph_en_masse(
        models: Dict[str, np.ndarray],
        out_dir: Path,
        reduction: str,  # 'PCA', 'TSNE', or 'both'
        words: List[GroundedWord],
        # hues: Union[List[float], List[int]],
        # sizes: List[int],
        perplexity: Optional[int] = None,
        categorical: bool = False
        ) -> None:
    Path.mkdir(out_dir, parents=True, exist_ok=True)
    word_ids = np.array([w.word_id for w in words])
    for model_name, embed in tqdm(models.items()):
        space = embed[word_ids]
        if reduction == 'PCA':
            visual = PCA(n_components=2).fit_transform(space)
        elif reduction == 'TSNE':
            assert perplexity is not None
            visual = TSNE(
                perplexity=perplexity, learning_rate=10,
                n_iter=5000, n_iter_without_progress=1000).fit_transform(space)
        elif reduction == 'both':
            assert perplexity is not None
            space = PCA(n_components=30).fit_transform(space)
            visual = TSNE(
                perplexity=perplexity, learning_rate=10,
                n_iter=5000, n_iter_without_progress=1000).fit_transform(space)
        else:
            raise ValueError('unknown dimension reduction method')
        if categorical:
            plot_categorical(visual, words, out_dir / f'{model_name}.png')
        else:
            plot(visual, words, out_dir / f'{model_name}.png')


In [ ]:
cherry_words = [
    'government', 'washington',
    'estate_tax', 'death_tax',
    'public_option', 'governmentrun',
    'foreign_trade', 'international_trade',
    'cut_taxes', 'trickledown'
]

cherry_words = [GroundedWord(w) for w in cherry_words]

In [8]:
base_dir = Path('../../results/SGNS deno/sans recomposer')
deno_space = load_decomposers_en_masse(base_dir, patterns='*/epoch10.pt')

Loading ../../results/SGNS deno/sans recomposer/L4 BS4/epoch10.pt
Loading ../../results/SGNS deno/sans recomposer/L4/epoch10.pt



/home/webson/Research/congressional_adversary/congressional_env/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'decomposer.Decomposer' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [9]:
models = deno_space
stuff = polarization

graph_en_masse(
    models, out_dir=base_dir / 'cherry/topic/t-SNE p5',
    reduction='TSNE', perplexity=5, words=stuff, categorical=True)
graph_en_masse(
    models, out_dir=base_dir / 'cherry/topic/t-SNE p3',
    reduction='TSNE', perplexity=3, words=stuff, categorical=True)
graph_en_masse(
    models, out_dir=base_dir / 'cherry/topic/t-SNE p2',
    reduction='TSNE', perplexity=2, words=stuff, categorical=True)

In [ ]:
models = cono_space

graph_en_masse(
    models, out_dir=base_dir / 'cherry/party/t-SNE p5',
    reduction='TSNE', perplexity=5, words=cherry_words, categorical=False)

graph_en_masse(
    models, out_dir=base_dir / 'cherry/party/t-SNE p3',
    reduction='TSNE', perplexity=3, words=cherry_words, categorical=False)

graph_en_masse(
    models, out_dir=base_dir / 'cherry/party/t-SNE p2',
    reduction='TSNE', perplexity=2, words=cherry_words, categorical=False)

In [ ]:
models = deno_space

graph_en_masse(
    models,
    out_dir=f'{base_dir}/decomposed deno/party/t-SNE p25',
    reduction='TSNE', perplexity=25,
    word_ids=J_ids, words=J_words, hues=J_skew, sizes=J_freq)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/decomposed deno/party/t-SNE p50',
    reduction='TSNE', perplexity=50,
    word_ids=J_ids, words=J_words, hues=J_skew, sizes=J_freq)

In [ ]:
models = cono_space

graph_en_masse(
    models, out_dir=f'{base_dir}/decomposed cono/topic/t-SNE p5',
    reduction='TSNE', perplexity=5,
    word_ids=J_ids, words=J_words, hues=J_deno, sizes=J_freq,
    categorical=True)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/decomposed cono/topic/t-SNE p3',
    reduction='TSNE', perplexity=3,
    word_ids=J_ids, words=J_words, hues=J_deno, sizes=J_freq,
    categorical=True)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/decomposed cono/topic/t-SNE p10',
    reduction='TSNE', perplexity=10,
    word_ids=J_ids, words=J_words, hues=J_deno, sizes=J_freq,
    categorical=True)

In [ ]:
models = cono_space

# graph_en_masse(
#     models,
#     out_dir=f'{base_dir}/Joint/topic/PCA',
#     reduction='PCA',
#     word_ids=J_ids, words=J_words, hues=J_skew, sizes=J_freq)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/decomposed cono/party/t-SNE p25',
    reduction='TSNE', perplexity=25,
    word_ids=J_ids, words=J_words, hues=J_skew, sizes=J_freq)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/decomposed cono/party/t-SNE p50',
    reduction='TSNE', perplexity=50,
    word_ids=J_ids, words=J_words, hues=J_skew, sizes=J_freq)

# Homogeneity V-Measure

In [ ]:
# Deno space, eval deno, higher is better
for model_name, model in deno_space.items():
    cluster_labels, true_labels = NN_cluster_ids(
        model, J_ids, categorical=True, top_k=10)    
    homogeneity, completeness, v_measure = np.around(
        homogeneity_completeness_v_measure(true_labels, cluster_labels), 4)
    print(model_name, homogeneity, completeness, v_measure, sep='\t')
#     print(pred_labels)

In [ ]:
# Deno space, eval cono, lower is better
for model_name, model in deno_space.items():
    cluster_labels, true_labels = NN_cluster_ids(
        model, J_ids, categorical=False, top_k=5)    
    homogeneity, completeness, v_measure = np.around(
        homogeneity_completeness_v_measure(true_labels, cluster_labels), 4)
    print(model_name, homogeneity, completeness, v_measure, sep='\t')
#     print(pred_labels)

In [ ]:
# Cono space, eval cono, higher is better
for model_name, model in cono_space.items():
    cluster_labels, true_labels = NN_cluster_ids(
        model, J_ids, categorical=False, top_k=5)    
    homogeneity, completeness, v_measure = np.around(
        homogeneity_completeness_v_measure(true_labels, cluster_labels), 4)
    print(model_name, homogeneity, completeness, v_measure, sep='\t')

In [ ]:
# Cono space, eval deno, lower is better
for model_name, model in cono_space.items():
    cluster_labels, true_labels = NN_cluster_ids(
        model, J_ids, categorical=True, top_k=5)    
    homogeneity, completeness, v_measure = np.around(
        homogeneity_completeness_v_measure(true_labels, cluster_labels), 4)
    print(model_name, homogeneity, completeness, v_measure, sep='\t')